In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
# df = pd.read_csv(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Lexicon based study/political_fact_checker.csv")

# df.info()

In [ ]:
# df_fake = pd.read_csv("/content/drive/Shareddrives/FYP - knk/Datasets/ISOT/Fake.csv")
# df_true = pd.read_csv("/content/drive/Shareddrives/FYP - knk/Datasets/ISOT/True.csv")
# df_fake["statement"]=df_fake["text"]
# df_true["statement"]=df_true["text"]
# df_true["label"]=0
# df_fake["label"]=1
# df_true["target"]=df_true["label"]
# df_fake["target"]=df_fake["label"]
# df=df_true.append(df_fake,ignore_index = True)
# df.drop(["subject","date"],axis=1 , inplace=True)
# df = df.sample(frac=1).reset_index(drop=True)
# print(df_true.info())
# print(df_fake.info())

In [ ]:
df_fake = pd.read_csv("/content/drive/Shareddrives/FYP - knk/Datasets/FakeNewsNet/gossipcop_fake.csv")
df_true = pd.read_csv("/content/drive/Shareddrives/FYP - knk/Datasets/FakeNewsNet/gossipcop_real.csv")
df_fake["statement"]=df_fake["title"]
df_true["statement"]=df_true["title"]
df_true["label"]=0
df_fake["label"]=1
df_true["target"]=df_true["label"]
df_fake["target"]=df_fake["label"]
df=df_true.append(df_fake,ignore_index = True)
# df.drop(["subject","date"],axis=1 , inplace=True)
df = df.sample(frac=1).reset_index(drop=True)
print(df_true.info())
print(df_fake.info())

In [ ]:
# df=pd.read_csv("/content/drive/Shareddrives/FYP - knk/Datasets/WELFake/sementic+sentiment_welfake.csv")
# df.info()

In [5]:
df.head()

,title,text,statement,label,target
0,OBAMA FIGHTS TO KEEP RADICAL AGENDA ALIVE: Ask...,This is a good reminder of how important it is...,This is a good reminder of how important it is...,1,1
1,Spanish prosecutor asks for Catalan police chi...,MADRID (Reuters) - Spain s state prosecutor ha...,MADRID (Reuters) - Spain s state prosecutor ha...,0,0
2,Former intelligence officials say Trump is bei...,WASHINGTON (Reuters) - Two former top U.S. int...,WASHINGTON (Reuters) - Two former top U.S. int...,0,0
3,LIVE STREAM: PRESIDENT TRUMP’S Historic Speech...,President Trump is scheduled to deliver a hist...,President Trump is scheduled to deliver a hist...,1,1
4,"Italy's 5-Star, stung by fake news claims, cal...",ROME (Reuters) - Italy s anti-establishment 5-...,ROME (Reuters) - Italy s anti-establishment 5-...,0,0


In [ ]:
# df["statement"]=df["text"]
# df["target"]=df["label"]

In [6]:
df = df.dropna(subset = ['statement'], how='all')

In [6]:
# df['statement'][225]

'Wearing face masks may cause "pulmonary fibrosis" and other health risks in children.'

In [7]:
# Remove Duplicates

duplicate_count = len(df['statement'])-len(df['statement'].drop_duplicates())
print('Duplicate count:', duplicate_count)
print('Total records before remove duplicates:', df.shape[0])

# drop duplicates (keep the last post of each of the duplicates)

df = df.drop_duplicates(subset='statement', keep="first")
print('updated record count:', df.shape[0])

Duplicate count: 6252
Total records before remove duplicates: 44898
updated record count: 38646


In [8]:
import string
import re

def lower(text):
  return text.lower()

# remove urls
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_nonascii(sent):
  return "".join([i for i in sent if i.isascii()])

def remove_punctuations(text):
  # text = text.translate(str.maketrans('', ' ', string.punctuation))
  # return text

  res = re.sub(r'[^\w\s]', ' ', text)
  return res

def remove_num(text):
  return "".join([c for c in text if not c.isdigit()])

# remove multiple spaces
def remove_mul_space(text):
  return " ".join(text.split())

def clean(text):
  text = lower(text)
  text = remove_urls(text)
  text = remove_nonascii(text)
  text = remove_punctuations(text)
  text = remove_num(text)
  text = remove_mul_space(text)
  return text

In [9]:
df['text'] = df['statement'].apply(clean)

In [10]:
df.tail()

,title,text,statement,label,target
44892,WATCH: HOLLYWOOD ACTOR SCOTT BAIO…All In For T...,add one more name to the growing list of celeb...,Add one more name to the growing list of celeb...,1,1
44893,JESSE WATTERS CALLS OUT Liberal Pundit For Rac...,jessebwatters what specifically are you fearfu...,.@jessebwatters: What specifically are you fe...,1,1
44895,Obama confident U.S. will move in right direct...,milan reuters former u s president barack obam...,MILAN (Reuters) - Former U.S. President Barack...,0,0
44896,Xi and Trump discuss sanctions pressure on Nor...,beijing washington reuters chinese president x...,BEIJING/WASHINGTON (Reuters) - Chinese Preside...,0,0
44897,Trump's dilemma: slower job growth or rising r...,washington reuters a drop in the u s unemploym...,WASHINGTON (Reuters) - A drop in the U.S. unem...,0,0


In [11]:
# Next we remove stop words.
# We use the SMART stop word list here
# Initially we import the text file and create the list.

stop = []
with open("/content/drive/Shareddrives/FYP - knk/Resources/SMART_STOP_WORDS.txt", "r") as f:
  for word in f:
      # Here we remove the apostrophe as well
      stop.append((word.strip()).replace("'",""))

# Remove the words in 'stop' list
def remove_stop_words(content):
  tokens = content.split()
  stop_removed_tokens = [t for t in tokens if t not in stop]
  convert_to_string = " ".join(stop_removed_tokens)
  return convert_to_string

In [12]:
df['text'] = df['text'].apply(remove_stop_words)

In [13]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()

def nltk2wn_tag(nltk_tag):
  if nltk_tag.startswith('J'):
    return wordnet.ADJ
  elif nltk_tag.startswith('V'):
    return wordnet.VERB
  elif nltk_tag.startswith('N'):
    return wordnet.NOUN
  elif nltk_tag.startswith('R'):
    return wordnet.ADV
  else:                    
    return None

def lemmatize_sentence(sentence):
  nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))    
  wn_tagged = map(lambda x: (x[0], nltk2wn_tag(x[1])), nltk_tagged)
  res_words = []
  for word, tag in wn_tagged:
    if tag is None:                        
      res_words.append(word)
    else:
      res_words.append(lemmatizer.lemmatize(word, tag))
  return " ".join(res_words)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from tqdm.auto import tqdm
tqdm.pandas()

In [14]:
df['text'] = df['text'].progress_apply(lemmatize_sentence)

In [15]:
df['target'].unique()

array([1, 0])

In [16]:
df['target']

0        1
1        0
2        0
3        1
4        0
        ..
44892    1
44893    1
44895    0
44896    0
44897    0
Name: target, Length: 38646, dtype: int64

In [17]:
# df_fake = df.loc[df['target'].isin(['false','barely-true','pants-fire'])]

# df_true = df.loc[df['target'].isin(['true','mostly-true','half-true'])]
df_fake = df.loc[df['target']==1]

df_true = df.loc[df['target']==0]

In [18]:
# df_fake["target"].replace({"false":1,'barely-true':1,'pants-fire':1}, inplace=True)
# df_true["target"].replace({'mostly-true':0,'half-true':0,"true":0}, inplace=True)

# dff=dff.loc[(dff['target']==1) | (dff['target']==0)]
# dff["target"] = pd.to_numeric(dff["target"])
# dff["target"].unique()

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [18]:
print(df_fake.info())
print(df_true.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17455 entries, 0 to 44893
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      17455 non-null  object
 1   text       17455 non-null  object
 2   statement  17455 non-null  object
 3   label      17455 non-null  int64 
 4   target     17455 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 818.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 21191 entries, 1 to 44897
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      21191 non-null  object
 1   text       21191 non-null  object
 2   statement  21191 non-null  object
 3   label      21191 non-null  int64 
 4   target     21191 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 993.3+ KB
None


In [19]:
df_fake_train, df_fake_test = train_test_split(df_fake, test_size=0.2, random_state=142)
df_true_train, df_true_test = train_test_split(df_true, test_size=0.2, random_state=142)


# df_fake_train = df_fake[:fake_length]
# df_fake_test = df_fake[fake_length:]

# df_true_train = df_true[:true_length]
# df_true_test = df_true[true_length:]

In [20]:
df_fake_train

,title,text,statement,label,target
43024,BREAKING: CHALLENGE OVER! Pennsylvania Recount...,pennsylvania legal document jill stein campaig...,"Pennsylvania According to legal documents, J...",1,1
7201,Trump Junior TOTALLY Staged This Awkward Phot...,week donald trump junior focus senate investig...,"Last week, Donald Trump Junior became a new fo...",1,1
37926,Karma In All Its Glory: Republican Chairwoman...,month ago linda sorenson republican chair delt...,"A little over a month ago, Linda Sorenson, the...",1,1
33219,Trial By YouTube: Mainstream Media Use Second-...,patrick henningsen st century wirethere exist ...,Patrick Henningsen 21st Century WireThere exi...,1,1
34534,"CASTING CRISIS: Orlando’s Actors, Agents and C...",shawn helton st century wiretruth stranger fic...,Shawn Helton 21st Century WireTruth is often s...,1,1
...,...,...,...,...,...
10623,"WATCH: TRUMP’S BI-RACIAL, FORMER GIRLFRIEND Of...",kara young biracial model date donald trump br...,"Kara Young, the biracial model who dated Donal...",1,1
15842,WATCH: We Found Donald Trump’s Campaign Theme...,back fox madtv air actress singer name nicole ...,"Way back in 1998, when Fox s MadTV was still o...",1,1
29903,Revealed: Loretta Lynch Given Talking Points f...,st century wire surface current event election...,"21st Century Wire says Clearly, below the surf...",1,1
15694,WATCH: HARRY REID Caught Calling Benghazi Moth...,harry reid disrespectful comment leftist democ...,Harry Reid s disrespectful comments are just a...,1,1


In [ ]:
# from collections import Counter

# fake = ' '.join(df_fake_train['text'].tolist())
# fake = fake.split()

# counts_fake = dict(Counter(fake))

# true = ' '.join(df_true_train['text'].tolist())
# true = true.split()

# counts_true = dict(Counter(true))

In [ ]:
# true_freq, fake_freq = 0, 0

# for key in counts_true.keys():
#   true_freq += counts_true[key]

# for key in counts_fake.keys():
#   fake_freq += counts_fake[key]

# for key in counts_true.keys():
#   counts_true[key] = counts_true[key]/float(true_freq)

# for key in counts_fake.keys():
#   counts_fake[key] = counts_fake[key]/float(fake_freq)

In [21]:
import pickle

# lexicon = {
#     'fake' : counts_fake,
#     'true' : counts_true,
#     'common': counts_common
# }


with open('/content/drive/Shareddrives/[FYP] Fake News Detection/Lexicon based study/WELFake_Lexicon1.pickle', 'rb') as handle:
    b = pickle.load(handle)

counts_fake=b["fake"]
counts_true=b["true"]
counts_common=b["common"]
counts_true_unique=b["unique_true"]
counts_fake_unique=b["unique_fake"]

In [ ]:
# print(counts_fake)
# print(counts_true)

{'travis': 0.00013429849182793678, 'county': 0.001625011751118035, 'resident': 0.0005103342689461597, 'trust': 0.00025516713447307985, 'tom': 0.0004834745705805724, 'delay': 0.0002417372852902862, 'win': 0.0011549670297202562, 'year': 0.00975007050670821, 'million': 0.005049623292730423, 'mark': 0.0005371939673117471, 'zuckerberg': 4.028954754838103e-05, 'third': 0.00017458803937631782, 'student': 0.0010743879346234943, 'cory': 8.057909509676206e-05, 'booker': 6.714924591396839e-05, 'newark': 0.00010743879346234942, 'graduate': 0.00017458803937631782, 'aclu': 9.400894427955574e-05, 'atheist': 6.714924591396839e-05, 'denver': 6.714924591396839e-05, 'demand': 0.00032231638038704826, 'nfl': 0.00018801788855911147, 'stop': 0.000832650649333208, 'tim': 0.0002820268328386672, 'tebow': 4.028954754838103e-05, 'pray': 8.057909509676206e-05, 'sideline': 4.028954754838103e-05, 'shut': 0.00044318502303219135, 'amtrak': 1.3429849182793678e-05, 'northeast': 2.6859698365587356e-05, 'corridor': 4.0289

In [ ]:
# test_fake = df_fake_test['text'].tolist()

# fake_count = 0

# for item in test_fake:
#   fake_score, true_score = 0, 0
#   for word in item.split():
#     if word in counts_true.keys():
#       true_score += counts_true[word]
#     if word in counts_fake.keys():
#       fake_score += counts_fake[word]
#   if fake_score > true_score: 
#     fake_count += 1    

In [ ]:
# print(fake_count)
# print(len(test_fake))

In [ ]:
# test_true = df_true_test['text'].tolist()

# true_count = 0

# for item in test_true:
#   fake_score, true_score = 0, 0
#   for word in item.split():
#     if word in counts_true.keys():
#       true_score += counts_true[word]
#     if word in counts_fake.keys():
#       fake_score += counts_fake[word]
#   if fake_score < true_score: 
#     true_count += 1  

In [ ]:
# print(true_count)
# print(len(test_true))

In [ ]:
# accuracy = (true_count+fake_count)/float(len(test_true)+len(test_fake)) * 100
# print(accuracy)

In [22]:
def generate_fake_score(text):
  fake_score, true_score , common_score ,unique_true_score,unique_fake_score = 0, 0 ,0 ,0 ,0
  for word in text.split():
    if word in counts_true.keys():
      true_score += counts_true[word]
    if word in counts_common.keys():
      common_score += counts_common[word]
    if word in counts_fake.keys():
      fake_score += counts_fake[word]
    if word in counts_true_unique.keys():
      unique_true_score+=counts_true_unique[word]
    if word in counts_fake_unique.keys():
      unique_fake_score+=counts_fake_unique[word]
  return pd.Series([fake_score, true_score , common_score ,unique_true_score,unique_fake_score], index=['fake_score', 'true_score', 'common_score',"unique_true_score","unique_fake_score"])

In [23]:
# generate_fake_score("Portland protesters were doing the usual idiotic tactic of blocking the street and not letting traffic proceed What they didn t realize is that the police aren t taking this anymore Watch the takedown of these punks Awesome!Check out the lady with the thumbs up! Haha!")

fake_score           0.004613
true_score           0.002909
common_score         0.000906
unique_true_score    0.000009
unique_fake_score    0.000011
dtype: float64

In [26]:
# text = "One of the world's strictest lockdowns is lifting."
# text = clean(text)
# text = remove_stop_words(text)
# text = lemmatize_sentence(text)

In [27]:
# generate_fake_score(text)

fake_score           0.001982
true_score           0.001439
common_score         0.000417
unique_true_score    0.000000
unique_fake_score    0.000000
dtype: float64

In [24]:
dff=df_fake_train["text"].apply(generate_fake_score)
df_fake_train=pd.concat([df_fake_train,dff],axis=1)

dff=df_fake_test["text"].apply(generate_fake_score)
df_fake_test=pd.concat([df_fake_test,dff],axis=1)

dff=df_true_train["text"].apply(generate_fake_score)
df_true_train=pd.concat([df_true_train,dff],axis=1)

dff=df_true_test["text"].apply(generate_fake_score)
df_true_test=pd.concat([df_true_test,dff],axis=1)

In [ ]:
# df_fake_train

In [ ]:
# dff=df["text"].apply(generate_fake_score)

In [ ]:
# dff=pd.concat([df,dff],axis=1)

In [36]:
df_train=df_fake_train.append(df_true_train,ignore_index=True).sample(frac=1).reset_index(drop=True)
df_test=df_fake_test.append(df_true_test,ignore_index=True).sample(frac=1).reset_index(drop=True)

# X_train=df_train[['fake_score', 'true_score', 'common_score',"unique_true_score","unique_fake_score"]]
# X_val=df_test[['fake_score', 'true_score', 'common_score',"unique_true_score","unique_fake_score"]]

X_train=df_train[['common_score',"unique_true_score","unique_fake_score"]]
X_val=df_test[['common_score',"unique_true_score","unique_fake_score"]]

y_train=df_train.target
y_val=df_test.target

In [ ]:
X_train

In [ ]:
# dff["target"].replace({"false":1,'barely-true':1,'pants-fire':1,'mostly-true':0,'half-true':0,"true":0}, inplace=True)
# dff=dff.loc[(dff['target']==1) | (dff['target']==0)]
# dff["target"] = pd.to_numeric(dff["target"])
# dff["target"].unique()


In [ ]:
# X=dff[['fake_score', 'true_score', 'common_score']]

In [ ]:
# Y=dff.target

In [ ]:
# X.head()

In [ ]:
# X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=142,stratify=Y)

In [27]:
import torch
import time
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, precision_recall_fscore_support, \
    confusion_matrix
from sklearn.svm import SVC
import re
import matplotlib.pyplot as plt

In [28]:
def compute_metrics(pred,ground_labels):
    labels_all = ground_labels
    preds_all = list(pred)
    
    precision, recall, f1, _ = precision_recall_fscore_support(labels_all, preds_all)
    acc = accuracy_score(labels_all, preds_all)
    confusion_mat = confusion_matrix(labels_all, preds_all)
    # tn, fp, fn, tp = confusiton_mat.ravel()
    out_dict = {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'confusiton_mat': confusion_mat
    }
    return out_dict

# XG BOOST

In [33]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=9), n_estimators=120,algorithm="SAMME.R", learning_rate=0.0001)
ada_clf.fit(X_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                         class_weight=None,
                                                         criterion='gini',
                                                         max_depth=9,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort='deprecated',
                          

In [34]:
from sklearn import metrics
predicted_y = ada_clf.predict(X_val)
gg=compute_metrics(predicted_y,y_val)
d=gg
tn, fp, fn, tp = d["confusiton_mat"].ravel()
print("{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}".format('prec-f','prec-t', 'rec-f','rec-t','f1-f','f1-t','accu','tn', 'fp', 'fn', 'tp'))
print ("{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:}\t{:}\t{:}\t{:}".format(d['precision'][0],d['precision'][1], d['recall'][0],d['recall'][1],d['f1'][0],d['f1'][1],d['accuracy'],tn, fp, fn, tp))

prec-f	prec-t	rec-f	rec-t	f1-f	f1-t	accu	tn	fp	fn	tp
0.549	0.576	0.392	0.720	0.457	0.640	0.567	708	1100	582	1496


# compare different models

In [37]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

In [38]:
classifiers=[['Logistic Regression :',LogisticRegression()],
       ['Decision Tree Classification :',DecisionTreeClassifier()],
       ['Gradient Boosting Classification :', GradientBoostingClassifier()],
       ['Ada Boosting Classification :',AdaBoostClassifier()],
       ['Extra Tree Classification :', ExtraTreesClassifier()],
       ['K-Neighbors Classification :',KNeighborsClassifier()],
       ['Support Vector Classification :',SVC()],
       ['Gaussian Naive Bayes :',GaussianNB()]]
cla_pred=[]

In [39]:
# ~ 4-5 min run 
for name,model1 in classifiers:
    print("-----------"+name+"-------------")
    model1.fit(X_train,y_train)
    predicted_y = model1.predict(X_val)
    score=compute_metrics(predicted_y,y_val)
    cla_pred.append(score)

-----------Logistic Regression :-------------
-----------Decision Tree Classification :-------------
-----------Gradient Boosting Classification :-------------
-----------Ada Boosting Classification :-------------
-----------Extra Tree Classification :-------------
-----------K-Neighbors Classification :-------------
-----------Support Vector Classification :-------------
-----------Gaussian Naive Bayes :-------------


In [40]:
#Prining the evaluation matrix to the console
# print("Summary\n Class 0- True news \n Class 1 - False news\n")
# for i in range(len(classifiers)):
#   print("-----------"+classifiers[i][0]+"-------------")
#   d=cla_pred[i]
#   print ("{:}\t{:}\t{:}\t{:}".format('label','precision','recall',"f1-score"))
#   print ("{:}\t{:.3f}\t\t{:.3f}\t{:.3f}".format("0", d['precision'][0], d['recall'][0],d['f1'][0]))
#   print ("{:}\t{:.3f}\t\t{:.3f}\t{:.3f}".format("1", d['precision'][1], d['recall'][1],d['f1'][1]))
#   print("\n")
#   print ("{:<5} {:<15}".format("accuracy", d['accuracy']))
#   print("\n")
#   print("confusiton_mat")
#   tn, fp, fn, tp = cla_pred[i]["confusiton_mat"].ravel()
#   print ("{:<25} {:<10} {:<10}".format("Actual \Prediction", "0","1"))
#   print ("{:<25} {:<10} {:<10}".format("0", tn,fp))
#   print ("{:<25} {:<10} {:<10}".format("1", fn,tp))
#   print("\n")

In [41]:
#Prining the evaluation matrix to the console
print("Summary\n Class 0- True news \n Class 1 - False news\n")
print("{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}".format('prec-f','prec-t', 'rec-f','rec-t','f1-f','f1-t','accu','tn', 'fp', 'fn', 'tp'))
for i in range(len(classifiers)):
  # print("-----------"+classifiers[i][0]+"-------------")
  d=cla_pred[i]
  tn, fp, fn, tp = cla_pred[i]["confusiton_mat"].ravel()
  print ("{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:}\t{:}\t{:}\t{:}".format(d['precision'][0],d['precision'][1], d['recall'][0],d['recall'][1],d['f1'][0],d['f1'][1],d['accuracy'],tn, fp, fn, tp))

Summary
 Class 0- True news 
 Class 1 - False news

prec-f	prec-t	rec-f	rec-t	f1-f	f1-t	accu	tn	fp	fn	tp
0.552	0.812	0.997	0.016	0.710	0.031	0.554	4226	13	3435	56
0.873	0.836	0.863	0.848	0.868	0.842	0.856	3660	579	531	2960
0.850	0.962	0.974	0.791	0.908	0.868	0.892	4130	109	728	2763
0.847	0.968	0.979	0.785	0.908	0.867	0.891	4148	91	752	2739
0.876	0.847	0.873	0.850	0.875	0.848	0.863	3702	537	524	2967
0.820	0.829	0.869	0.769	0.844	0.798	0.824	3685	554	808	2683
0.680	0.674	0.780	0.554	0.726	0.608	0.678	3306	933	1558	1933
0.727	0.983	0.992	0.547	0.839	0.703	0.791	4205	34	1581	1910


In [42]:
# --------Logistic Regression :-------------
# -----------Decision Tree Classification :-------------
# -----------Gradient Boosting Classification :-------------
# -----------Ada Boosting Classification :-------------
# -----------Extra Tree Classification :-------------
# -----------K-Neighbors Classification :-------------
# -----------Support Vector Classification :-------------
# -----------Gaussian Naive Bayes :-----------

# SVG classifier

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
start = time.time()

svm_classifier = SVC(probability=True)
svm_classifier.fit(X_train,y_train)

end = time.time()
process = round(end-start,2)
print("Support Vector Machine Classifier has fitted, this process took {} seconds".format(process))

# print(svm_classifier.score(X_val,y_val))

Support Vector Machine Classifier has fitted, this process took 167.94 seconds


In [ ]:
predicted_y = svm_classifier.predict(X_val)
print(predicted_y)

[1 0 0 ... 0 0 0]


In [ ]:
gg=compute_metrics(predicted_y,y_val)

In [ ]:
gg

{'accuracy': 0.6685640362225097, 'confusiton_mat': array([[3656,  583],
        [1979, 1512]]), 'f1': array([0.74053069, 0.54135338]), 'precision': array([0.64880213, 0.72171838]), 'recall': array([0.86246756, 0.43311372])}

In [ ]:
d=gg
tn, fp, fn, tp = d["confusiton_mat"].ravel()
print("{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}\t{:}".format('prec-f','prec-t', 'rec-f','rec-t','f1-f','f1-t','accu','tn', 'fp', 'fn', 'tp'))
print ("{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:}\t{:}\t{:}\t{:}".format(d['precision'][0],d['precision'][1], d['recall'][0],d['recall'][1],d['f1'][0],d['f1'][1],d['accuracy'],tn, fp, fn, tp))

prec-f	prec-t	rec-f	rec-t	f1-f	f1-t	accu	tn	fp	fn	tp
0.649	0.722	0.862	0.433	0.741	0.541	0.669	3656	583	1979	1512


# RESULTS

In [ ]:
# --------Logistic Regression :-------------
# -----------Decision Tree Classification :-------------
# -----------Gradient Boosting Classification :-------------
# -----------Ada Boosting Classification :-------------
# -----------Extra Tree Classification :-------------
# -----------K-Neighbors Classification :-------------
# -----------Support Vector Classification :-------------
# -----------Gaussian Naive Bayes :-----------

# Cross Validated against WELFake Lexicon

In [ ]:
# ISOT
# Summary
#  Class 0- True news 
#  Class 1 - False news

# prec-f	prec-t	rec-f	  rec-t	  f1-f	  f1-t	  accu	  tn	  fp	  fn	  tp
# 0.839	  0.853	  0.888	  0.793	  0.863	  0.822	  0.845	  3764	475	  724	  2767
# 0.830	  0.791	  0.827	  0.795	  0.828	  0.793	  0.812 	3504	735	  717	  2774
# 0.866	  0.860	  0.888	  0.833	  0.877	  0.846	  0.863	  3765	474	  582	  2909
# 0.824	  0.852	  0.890	  0.770	  0.856	  0.809	  0.836	  3774	465	  804	  2687
# 0.858	  0.836	  0.866	  0.825	  0.862 	0.830	  0.848	  3672	567	  610	  2881
# 0.857	  0.844	  0.874	  0.824	  0.866	  0.833	  0.851	  3706	533	  616	  2875
# 0.869	  0.862	  0.890	  0.837	  0.879	  0.849	  0.866	  3771	468	  570	  2921
# 0.585	  0.600	  0.854	  0.265	  0.695	  0.367	  0.588	  3622	617	  2567	924

In [ ]:
# GossipCop
# Summary
#  Class 0- True news 
#  Class 1 - False news

# prec-f	prec-t	rec-f	  rec-t	  f1-f	  f1-t	  accu	  tn	  fp	  fn	  tp
# 0.767	  0.000	  1.000	  0.000	  0.868	  0.000	  0.767	  3181 	0	    968	  0
# 0.765	  0.229 	0.757	  0.238	  0.761	  0.233	  0.636	  2407	774	  738	  230
# 0.769	  0.526	  0.994	  0.021	  0.868	  0.040	  0.767	  3163	18	  948	  20
# 0.770	  0.568	  0.994	  0.026	  0.868	  0.049	  0.768	  3162	19	  943	  25
# 0.767	  0.239	  0.886	  0.118	  0.822	  0.158	  0.707	  2818	363	  854	  114
# 0.770	  0.267	  0.903	  0.116	  0.832	  0.161	  0.719	  2873	308	  856	  112
# 0.770	  0.605 	0.995	  0.024	  0.868	  0.046	  0.769	  3166	15  	945	  23
# 0.771	  0.324	  0.963	  0.059	  0.856	  0.100	  0.752	  3062	119	  911	  57

In [ ]:
# Politifact
# Summary
#  Class 0- True news 
#  Class 1 - False news

# prec-f	prec-t	rec-f	  rec-t	  f1-f	  f1-t	  accu	  tn	  fp	  fn	  tp
# 0.000	  0.535	  0.000	  1.000	  0.000	  0.697	  0.535	  0	    1808  0	    2078
# 0.470	  0.539	  0.480	  0.529	  0.475	  0.534	  0.506	  867	  941	  978	  1100
# 0.557	  0.585 	0.426	  0.705	  0.483	  0.640	  0.575	  770	  1038	613	  1465
# 0.566	  0.584	  0.397	  0.735	  0.467	  0.651	  0.578	  718	  1090	550	  1528
# 0.500	  0.565	  0.496	  0.569	  0.498	  0.567	  0.535	  897	  911	  896	  1182
# 0.485	  0.550	  0.462	  0.573	  0.473	  0.561	  0.521	  836	  972	  888	  1190
# 0.586	  0.578	  0.331	  0.796	  0.423	  0.670	  0.580	  599	  1209	423	  1655
# 0.559	  0.574	  0.357	  0.755	  0.436	  0.652	  0.570	  646	  1162	510	  1568

# Cross Validated with Politifact

In [ ]:
# ISOT
# Summary
#  Class 0- True news 
#  Class 1 - False news

# prec-f	prec-t	rec-f	  rec-t	  f1-f	  f1-t	  accu	  tn	  fp	  fn	  tp
# 0.631	  0.689	  0.854	  0.394	  0.725	  0.501	  0.646	  3618	621	  2117	1374
# 0.636	  0.561	  0.643	  0.553 	0.640	  0.557	  0.602	  2726	1513	1560	1931
# 0.672	  0.699	  0.817	  0.516	  0.737	  0.593	  0.681	  3463	776	  1691	1800
# 0.672	  0.686	  0.802	  0.526	  0.732	  0.595	  0.677	  3400	839	  1656	1835
# 0.659	  0.620	  0.725	  0.544	  0.690	  0.579	  0.643	  3075	1164	1593	1898
# 0.658	  0.616	  0.720	  0.546	  0.687	  0.579	  0.641	  3050	1189	1585	1906
# 0.649	  0.722	  0.862	  0.433	  0.741	  0.541	  0.669	  3656	583 	1979	1512
# 0.549	  1.000	  1.000	  0.001	  0.708	  0.001	  0.549	  4239	0	    3489	2